In [1]:
import torch
from lightning_datasets import MnistDataModule
import pytorch_lightning as L
from torch import nn
from torchmetrics import Accuracy
from pytorch_lightning.loggers import MLFlowLogger
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
class LeNetWithBatchNormalization(L.LightningModule):
    def __init__(self, learning_rate: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 6, 5, padding=2),
            nn.BatchNorm2d(6),
            nn.Sigmoid(),
            nn.AvgPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.AvgPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(400, 120),
            nn.BatchNorm1d(120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
        self.loss = nn.CrossEntropyLoss()
        self.lr = learning_rate
        self.accuracy = Accuracy(task='multiclass', num_classes=10)

    def forward(self, X):
        return self.net(X)

    def training_step(self, batch, batch_idx):
        loss, output, y = self.__common_step(batch, batch_idx)
        accuracy = self.accuracy(output, y)
        self.logger.log_metrics({"training_loss": loss, "train_acc": accuracy})
        return loss

    def validation_step(self, batch, batch_idx):
        loss, output, y = self.__common_step(batch, batch_idx)
        accuracy = self.accuracy(output, y)
        self.logger.log_metrics(
            {"validation_loss": loss, "validation_acc": accuracy})
        return loss

    def test_step(self, batch, batch_idx):
        loss, output, y = self.__common_step(batch, batch_idx)
        accuracy = self.accuracy(output, y)
        self.logger.log_metrics({"test_loss": loss, "test_acc": accuracy})
        return loss

    def __common_step(self, batch, batch_idx):
        X, y = batch
        output = self(X)
        loss = self.loss(output, y)
        return loss, output, y

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.lr)

In [3]:
model = LeNetWithBatchNormalization(1.0)
dataset = MnistDataModule(image_size=(28, 28), batch_size=256)
logger = MLFlowLogger(run_name="LeNet with BatchNorm")
trainer = L.Trainer('gpu', logger=logger, max_epochs=10)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model, dataset)

You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti SUPER') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | net      | Sequential         | 62.2 K | train
1 | loss     | CrossEntropyLoss   | 0      | train
2 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
62.2 K    Trainable params
0         Non-trainable params
62.2 K    Total params
0.249     Total estimated model params size (MB)
19        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [5]:
trainer.validate(model, dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

[{}]

In [6]:
trainer.test(model, dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[{}]